In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
from IPython.display import Image, display

import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", """Anda adalah asisten yang membantu. Jawab semua pertanyaan sebaik
        kemampuan Anda."""),
    ("placeholder", "{messages}"),
])

 

chain = prompt | llm | StrOutputParser()

chain.invoke({
    "messages": [
        ("human","""Terjemahkan kalimat ini dari Bahasa Inggris ke Prancis: I like
            programing."""),
        ("ai", "J'adore programmer."),
        ("human", "Apa yang baru saja Anda katakan?"),
    ],
})

'Saya baru saja menerjemahkan kalimat "I like programming" dari Bahasa Inggris ke Prancis, yang dalam hal ini menjadi "J\'adore programmer." Artinya adalah "Saya suka memprogram."'

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages 


class State(TypedDict):
    # Pesan memiliki tipe "list". Fungsi `add_messages`
    # dalam anotasi menentukan bagaimana status ini seharusnya
    # diperbarui (dalam kasus ini, ia menambahkan pesan baru ke
    # list, daripada mengganti pesan sebelumnya)
	messages: Annotated[list, add_messages]

builder = StateGraph(State)
def chatbot(state: State):
    answer = llm.invoke(state["messages"])
    return {"messages": [answer]}

# Argumen pertama adalah nama simpul unik
# Argumen kedua adalah fungsi atau Runnable yang akan dijalankan
builder.add_node("chatbot", chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

graph = builder.compile()
png_bytes =graph.get_graph().draw_mermaid_png()
display(Image(png_bytes))


In [ ]:
input = {"messages": [HumanMessage('hi!')]}
for chunk in graph.stream(input):
    print(chunk)


In [28]:
def print_messages(result, last_n=None):
    """
    Fungsi untuk menampilkan pesan.
    Jika last_n diisi (misal: 2), hanya menampilkan n pesan terakhir.
    """
    messages = result.get("messages", [])
    
    # Jika last_n diberikan, kita slice list-nya (misal: messages[-2:])
    if last_n is not None:
        messages = messages[-last_n:]
    
    for msg in messages:
        role = "😎 USER" if msg.type == "human" else "🤖 BOT"
        print(f"{role}: {msg.content}")

In [29]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    # Pesan memiliki tipe "list". Fungsi `add_messages`
    # dalam anotasi menentukan bagaimana status ini seharusnya
    # diperbarui (dalam kasus ini, ia menambahkan pesan baru ke
    # list, daripada mengganti pesan sebelumnya)
	messages: Annotated[list, add_messages]

builder = StateGraph(State)
def chatbot(state: State):
    answer = llm.invoke(state["messages"])
    return {"messages": [answer]}

# Argumen pertama adalah nama simpul unik
# Argumen kedua adalah fungsi atau Runnable yang akan dijalankan
builder.add_node("chatbot", chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

graph = builder.compile(checkpointer=MemorySaver())

thread_1 = {"configurable": {"thread_id": "1"}}
result_1 = graph.invoke(
    { "messages": [HumanMessage("hai, nama saya Jack!")] },
    thread_1
)
print_messages(result_1,last_n=2)
print("-----------------------------------------------------------------")

result_2 = graph.invoke(
    { "messages": [HumanMessage("siapa nama saya?")] },
    thread_1
)
print_messages(result_2,last_n=2)

😎 USER: hai, nama saya Jack!
🤖 BOT: Hai Jack! Senang bertemu denganmu 😊 Ada yang bisa saya bantu hari ini?
-----------------------------------------------------------------
😎 USER: siapa nama saya?
🤖 BOT: Nama kamu adalah Jack! 😄 Seperti yang kamu bilang tadi. Mau aku panggil sesuatu yang lain?
